In [1]:
import polars as pl
import os
import sys
os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"
sys.path.append("..")
%load_ext autoreload
%autoreload 2

In [25]:
from synthetic_data.tasks.writing import WritingScoreAnnotate, GutenbergBacktranslationFromTxt
from synthetic_data.utils import ldictl
gutenberg_pq: pl.DataFrame = pl.read_parquet("../dataset_files/epubs.parquet")

# input_df: pl.DataFrame = pl.read_parquet("../dataset_files/gutenberg_backtranslate.parquet")
task = GutenbergBacktranslationFromTxt()

# mock_batch = ldictl(input_df.head(4).to_dicts())
dataset = gutenberg_pq

In [29]:
from datasets import Dataset
dataset = Dataset.from_pandas(dataset.to_pandas())
sample_out = await task.preprocess_row(dataset[0])
sample_out = sample_out[0:16]
task_out = task.format_input_conversation(sample_out)

In [33]:
print(task_out[1][0]['content'])


You are tasked with creating an instruction for a large language model to recreate a paragraph from a book through a process called "back-translation." Here is the original paragraph:

<original_paragraph>
Earth barely resembles its former state. From ecologically sound
super-planet to smoking, mutated ruin, the mayhem has waned in the past
century. Habitable areas have not lost their lethality, nor have their
inhabitants diminished in number or hostility. What has changed is that
there are some areas in the world that have been settled and tamed by
various groups, factions, or cults that seek to reform the world
according to their agenda or creed.

After many decades of fighting and settling a safe area around their
newly fortified city Legacy-1, the Architechs created a system of
identifying the danger level of an area to help explorers navigate the
new world before them. These are called Alpha, Gamma, and Omega Zones.

Alpha Zones

An Alpha Zone is mostly cleared of deadly flora an

In [31]:
from dotenv import load_dotenv
load_dotenv("../.env")

[autoreload of synthetic_data.prompts failed: Traceback (most recent call last):
  File "/Users/brianfitzgerald/Documents/GitHub/smolmodels/.venv/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 276, in check
    superreload(m, reload, self.old_objects)
  File "/Users/brianfitzgerald/Documents/GitHub/smolmodels/.venv/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 475, in superreload
    module = reload(module)
             ^^^^^^^^^^^^^^
  File "/Users/brianfitzgerald/.local/share/uv/python/cpython-3.11.11-macos-aarch64-none/lib/python3.11/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 621, in _exec
  File "<frozen importlib._bootstrap_external>", line 936, in exec_module
  File "<frozen importlib._bootstrap_external>", line 1074, in get_code
  File "<frozen importlib._bootstrap_external>", line 1004, in source_to_code
  File "<frozen importlib._bootstrap>", line 241, 

True

In [34]:
from synthetic_data.generation import get_generation_wrapper


generation_wrapper = get_generation_wrapper("gemini-2.0-flash")
completions = await generation_wrapper.generate(
    task_out
)

2025-03-21 18:22:16.552 | INFO     | synthetic_data.generation:get_generation_wrapper:425 - Logging in to Hugging Face Hub
Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [36]:
output_rows_batch = task.format_output_rows(completions, sample_out)

In [37]:
print(output_rows_batch[0]['instruction'])

Write a paragraph detailing the backstory of a post-apocalyptic tabletop wargame setting. The tone should be both grandiose and foreboding, resembling a historical account delivered in a lecture. Describe a technologically advanced past civilization that brought about its own downfall through hubris and reliance on artificial intelligence. This AI, meant to terraform planets, was sabotaged, resulting in a cataclysmic event on Earth, mutating life and devastating the environment.

Mention factions or groups who survived, specifically those safeguarding ancient knowledge and technology, rebuilding civilization. Portray the world as a dangerous place, emphasizing the threats faced by those seeking to recover lost technology: rival factions, mutated creatures, and mercenary groups. Hint at a central organization or "order" that the player characters belong to, with the ultimate goal of restoring the world. Conclude by addressing the reader as a newcomer ("Apprentice") to this order, taskin

In [ ]:
import tiktoken
from synthetic_data.gutenberg_parser import super_cleaner
from synthetic_data.tasks.writing import find_valid_paragraph_chunks
from IPython.display import display, Markdown

tiktoken_encoder = tiktoken.get_encoding("o200k_base")

df_sampled = df.sample(100)
for row in df.iter_rows(named=True):
    text = row["text"]
    paragraphs = super_cleaner(text)

    # find chunks of consecutive paragraphs with dialogue and at least 3 sentences
    valid_chunks = find_valid_paragraph_chunks(paragraphs, tiktoken_encoder)
    display(Markdown(f"### {row['title']}"))
    for chunk in valid_chunks:
        display(Markdown(chunk[0]))

